</br>
<font size="12">PCA and population structure with called genotypes</font>


# PCA for wildebeest

Again we will use data from the Blue Wildebeest. To simply the the analysis we have included only on of the Brindles populations ( B-Etosha). 

<img src="https://raw.githubusercontent.com/popgenDK/popgenDK.github.io/gh-pages/images/slider/wildeBeastMap.png" alt="image info" />





# Setup enviroment

In [ ]:
COURSE_PATH=/course/popgen25
DATA_PATH=$COURSE_PATH/pca
SOFTWARE_PATH=$COURSE_PATH/software

echo --programs that are installed:--
type plink
type PCAone


#make folder 
FOLDER=~/popgen25_pca
echo -e "\n--creating folder-- "
echo $FOLDER
mkdir -p $FOLDER

# enter folder
cd $FOLDER

#make sym link for data and current folder
ln -sfn $FOLDER ~/current_folder
ln -sfn $DATA_PATH ~/data_folder

# Software and data

### data 
We will use the same wildebeest data as in the ADMIXTURE analysis. The genotype data is in plink format consisting of three files bim/bam/fam. For more details about the data see the admixture exercises

In [ ]:
# make links to files and add them to the folder
cp -sf ${DATA_PATH}/blue_wildebeest_thin* .
cp -r -sf  ${DATA_PATH}/multiRunK7 .

ls blue_wildebeest_thin*

echo -e "\n-- number of lines in bim file (number of SNPs) --"
wc -l blue_wildebeest_thin.bim

echo -e "\n-- number of lines in fam file (number of indivuals) --"
wc -l blue_wildebeest_thin.fam

echo -e "\n--counts of populations/subspecies from first column of fam file --"
cut -f1 -d" " blue_wildebeest_thin.fam | sort | uniq -c




### Software
We will be using PCAone for this exercise. PCAone works both for called genotypes and on genotype likelihoods. 

Let perform PCA on the whole data (without LD pruning). 

We will use PCAone first:



In [ ]:
PCAone -h

Shows the options. To perform the PCA use the following command

In [ ]:

PCAone -b blue_wildebeest_thin -o blue_wildebeest_thin

 - look at the above output. How many SNPs and how many individuals?
 
 The default is to calculate the top 10 PCs. If you want more you can use the option --pc <INT> to choose a different number. However, let see what the top PCs capture. 
    
First let look the two first PCs as well as the admixture proportions estimated in the prevous admixture exercises


In [ ]:
#read in code to plot admixture proportions ( plotAdmix function)
source("https://raw.githubusercontent.com/GenisGE/evalAdmix/master/visFuns.R")

options(repr.plot.width=12, repr.plot.height=12)
layout(matrix(c(1,1,2,3),nrow=2,by=T),height=c(2,4),width=2:1)

# Read in inferred admixture proportions
q <- read.table("~/current_folder/multiRunK7/blue_wildebeest_noLD.7.Q_4")

#read in the population labels (first column of fam file)
tab <- table(pop <- read.table("~/current_folder/blue_wildebeest_thin.fam")[,1])

#make the plot. 
plotAdmix(q,pop=pop,rotatelab=15,padj=0.15,cex.lab=1.4,col=c(3,5,8,4,2,6,7))


pca <- read.table("~/current_folder/blue_wildebeest_thin.eigvecs")
#layout(matrix(1:2,nrow=1),w=c(4,2))
plot(pca[,1:2],col=as.integer(as.factor(pop))+1,ylab=paste("PC",1),xlab=paste("PC",2),cex.lab=1.5,cex=2,lwd=8)
plot.new()
legend("top",legend=names(tab),bty="n",xpd=T,cex=2,text.col=1:length(tab)+1,text.font=2)


 - What information do you get from the PCA that you don't get from the ADMIXTURE results?
 - Can you identify the admixed individuals?
 
 
 Lets see what the other PCs show. 
 

In [ ]:
options(repr.plot.width=12, repr.plot.height=16)

par(mfrow=c(3,2))
for(pc in 0:4)
    plot(pca[,pc*2+1:2],col=as.factor(pop),ylab=paste("PC",pc*2+2),xlab=paste("PC",pc*2+1),cex.lab=1.5,cex=2,lwd=8)


 - How many PCs are used to separate the populations?
 - What do you think is captured on PC 7 and 8?
 - What  do you think is captures on PC 9 and 10?
 

## Bonus exercise if there is time


Lets try to run the PCA after pruning LD (linkage disequillibrium) from the data. The noLD data was created in the admixture exercise. 
Run first the pruning with PCA  and then run PCA on the pruned data

In [ ]:
PCAone -b blue_wildebeest_thin -k 6 --ld -o pcaone  

In [ ]:
#see created files
ls pcaone*

Calculate LD between markers in a 1Mb sliding window and identify a set of SNPs so that the LD is less than $r^2<0.1$

In [ ]:
PCAone -B pcaone.residuals \
         --match-bim pcaone.mbim \
         --ld-r2 0.1 \
         --ld-bp 1000000 \
         -o pcaone
         

  The LD measure has been adjusted for population structure using the PCs. 
  - Why does the LD need to be adjusted for population structure?
  - Why do we use a sliding window?
  
 Lets make a new plink file containing only the markers in low LD

In [ ]:
echo --number of variants to be keept --
wc -l pcaone.ld.prune.in

echo -e "\n --Extract variants using plink --"
plink --bfile blue_wildebeest_thin \
    --extract pcaone.ld.prune.in  \
    --make-bed  \
    --out blue_wildebeest_noLD  \
    --chr-set 29


We can not perform the PCA again on the pruned data

In [ ]:
PCAone  -b blue_wildebeest_noLD -o blue_wildebeest_noLD


We can start by comparing the singular values. These are proportional to the variance explained so that higher values means that the corresponding PC captures more information about the data. 

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)

singularValues <- scan("~/current_folder/blue_wildebeest_thin.eigvals")
singularValuesNoLD <- scan("~/current_folder/blue_wildebeest_noLD.eigvals")

varianceExplained <- function(x)
    x^2/sum(x^2)

varExp <- varianceExplained(rbind(singularValues,singularValuesNoLD))


barplot(varExp*100,beside=T,col=2:3,legend=c("With LD","Without LD"),
       ylab="%variance explained",xlab="PC 1:10")

 - which data set captures the most information about the population structure from the first top PCs?
 
 Lets plot the PCs from the two data sets

In [ ]:
options(repr.plot.width=12, repr.plot.height=32)

tab <- table(pop <- read.table("~/current_folder/blue_wildebeest_noLD.fam")[,1])


pcaNoLD <- read.table("~/current_folder/blue_wildebeest_noLD.eigvecs")


par(mfrow=c(5,2))
for(pc in 0:4){
 
    plot(pca[,pc*2+1:2],col=as.factor(pop),ylab=paste("PC",pc*2+2),xlab=paste("PC",pc*2+1),cex.lab=1.5,cex=2,lwd=8,main="with LD")
    plot(pcaNoLD[,pc*2+1:2],col=as.factor(pop),ylab=paste("PC",pc*2+2),xlab=paste("PC",pc*2+1),cex.lab=1.5,cex=2,lwd=8,main="No LD")

    
    }

 - Which of the PCs from prevous analysis capture LD and not population structure?
 - Do you think it is better to perform PCA with out without LD?

## Identity by state tree
As a last minute addition we can also make a neighbour joining tree by first computing identity-by-state distances between individuals which a just the proportion of sites between two individuals where they are different.
We can then load these distances into R and produce a NJ tree with the package APE.


In [ ]:
# get IBS distances with plink
plink --allow-extra-chr \
    --bfile blue_wildebeest_thin \
    --distance square 1-ibs \
    --chr-set 29 \
    --out plink


In [ ]:
library(ape)

# read in distances
m <- as.matrix(read.table("~/current_folder/plink.mdist", header = F))

# read id of individuals
id <- read.table("~/current_folder/plink.mdist.id")
rownames(m) <- id$V2
colnames(m) <- id$V2

pops <- c(4,6,8,2,5,7,3)
names(pops) <- unique(id$V1)

plot(nj(m), tip.color = pops[id$V1], type = "unrooted", show.tip.label = TRUE)
add.scale.bar()
legend("topright",
       legend = names(pops),
       fill = pops,
          ncol=4)

 - Can you find the admixed individual?
 - Try removing the " type = 'unrooted'," argument from the plotting command above and see what happens to the tree